In [1]:
from bauer.utils.data import load_garcia2022
from bauer.models import MagnitudeComparisonModel
import numpy as np
import pandas as pd
import seaborn as sns
from bauer.utils.plotting import plot_ppc
from utils import create_design_magJudge, simulate_choices_subind
from scipy.stats import pearsonr
import arviz as az
import os.path as op

In [ ]:
params = pd.read_csv('/Users/mrenke/git/numrisk/behavior_simulations/sub-params_Miguel.csv')
sub_params = pd.DataFrame(data = {'subject': params.iloc[:,0], 'n1_evidence_sd': params.iloc[:,1], 'n2_evidence_sd': params.iloc[:,2], 'n2_prior_mu':params.iloc[:,3]})

target_folder = '/Users/mrenke/git/numrisk/behavior_simulations/data_saved'
df = pd.DataFrame()

count = 0
for denominator in [6, 10, 14, 20, 26]:

    frac = np.linspace(-6,6, 13)
    frac = np.delete(frac, 6)                     
    fractions = np.power(2,(frac/denominator))

    df_task = create_design_magJudge(fractions, repetitions=3, n_runs= 6)

    df_ts_s =  pd.DataFrame() # df trial settings als subs
    for sub in sub_params['subject']:
        df_task['subject'] = np.array(sub, copy=len(df_task))
        df_ts_s = pd.concat([df_ts_s, df_task])

    for run in range(1,10):
        df_sim = simulate_choices_subind(df_ts_s, sub_params.set_index('subject'))
        df_sim['trial_nr'] = df_sim['trial']
        df_sim['choice'] = df_sim['sim_choice']
        df_sim = df_sim.set_index(['subject', 'trial_nr'])

        model = MagnitudeComparisonModel(df_sim)
        model.build_estimation_model()
        trace = model.sample(draws=1000, tune=1000)

        az.to_netcdf(trace, op.join(target_folder, f'denom{denominator}_run{run}_trace.netcdf'))
        df_sim.to_csv(op.join(target_folder, f'denom{denominator}_run{run}_sim-data.csv'))

        n1_ev_sd = trace.posterior['n1_evidence_sd'].to_dataframe()
        denom = n1_ev_sd
        dc = pd.DataFrame(data={'evid_sd_std': denom.groupby('subject').std()['n1_evidence_sd'],'evid_sd_mu':denom.groupby('subject').mean()['n1_evidence_sd'], 'true_evid_sd_mu': sub_params.set_index('subject')['n1_evidence_sd'] })

        x = 'evid_sd_mu'
        y = 'true_evid_sd_mu'
        z =  'evid_sd_std'

        ccy = pearsonr(dc[x], dc[y])
        ccz = pearsonr(dc[x], dc[z])

        count += 1
        d = pd.DataFrame(data={'denom' : denominator,'run': run, 
                 'evid_sd_mu_true-est_cc': round(ccy[0],3), 
                 'evid_sd_mu_true-est_pv': round(ccy[1],3), 
                 'evid_sd_mu-std_cc': round(ccz[0],3),    
                 'evid_sd_mu-std_pv': round(ccz[1],3),
                 'mean_evid_sd_std' : dc['evid_sd_std'].mean()},
                index=[count])

        df = pd.concat([df,d ])